In [12]:
#--------------------Theory Background--------------------#
print('Running Theory Section')

import numpy as np

# Parameters
vh = 200
vl = 100
ph = 160
pl =  80
ch =  40
cl =   0

m1 = (ph - vl) / (vh - vl)
m2 = (ph - pl) / (vh - vl)
kl = (pl - cl) / (ph - cl)
kh = (pl - ch) / (ph - ch)
x1 = 1 / (2 * m1)
x2 = 1 / (2 * m2)
x3 = 1 - x2

# function to find prob(ph) in a mixed equilibrium (binary search for a root)
def findx(n, k):
    def f(x):
        return (1 - k) * np.power(x, n) - np.power(x, n-1) + k
    lower_bound = np.zeros(len(n))
    upper_bound = (n - 1) / (n * (1 - k))
    while (~np.isclose(lower_bound, upper_bound)).any():
        midpoint    = (lower_bound + upper_bound) / 2
        f_midpoint  = f(midpoint)
        lower_bound = (f_midpoint >= 0) * midpoint + (f_midpoint < 0) * lower_bound
        upper_bound = (f_midpoint >= 0) * upper_bound + (f_midpoint < 0) * midpoint
    return midpoint

#--------------------Equilibrium Outcomes for each n--------------------#

precision = 100
n = np.linspace(1, 5, 2 * precision + 1)

#----------low-type mixes----------#

n_threshold1 = np.log(kl * x1 / (1 - x1 + kl * x1)) / np.log(x1)
n_threshold2 = np.log(kl * x2 / (1 - x2 + kl * x2)) / np.log(x2)

caseDindicator = (n <= n_threshold1)
caseD_x = np.array([x1 for i in n])
caseD_BH_BL_DB = np.array([0 for i in n])
caseD_BL_BH_DB = kl * (1 - np.power(x1, n)) / ((1 - x1) * np.power(x1, n - 1))
caseD_BL_DB_BH = 1 - caseD_BL_BH_DB

caseEindicator = (n >= n_threshold2)
caseE_x = np.array([x2 for i in n])
caseE_BH_BL_DB_numer = (kl * x2 * (1 - np.power(x2, n)) - (1 - x2) * np.power(x2, n))
caseE_BH_BL_DB_denom = ((1 - x2 + kl * x2) * (1 - np.power(x2, n) - np.power(1 - x2, n)))
caseE_BH_BL_DB = np.divide(caseE_BH_BL_DB_numer, caseE_BH_BL_DB_denom, where=caseE_BH_BL_DB_denom != 0)
caseE_BH_BL_DB[caseE_BH_BL_DB_denom == 0] = np.nan
caseE_BL_BH_DB = 1 - caseE_BH_BL_DB
caseE_BL_DB_BH = np.array([0 for i in n])

caseBindicator = (n < n_threshold1) & (n > n_threshold2)
caseB_x = findx(n, kl)
caseB_BH_BL_DB = np.array([0 for i in n])
caseB_BL_BH_DB = np.array([1 for i in n])
caseB_BL_DB_BH = np.array([0 for i in n])

# combining cases into one array for graphing
idxE = np.where(caseEindicator > 0)[0][0]
idxD = np.where(caseDindicator > 0)[0][-1]
lmix_n         = np.concatenate([n[:idxD + 1], n[idxE:idxD + 1][::-1], n[idxE:]])
lmix_x         = np.concatenate([caseD_x[:idxD + 1], caseB_x[idxE:idxD + 1][::-1], caseE_x[idxE:]])
lmix_BH_BL_DB  = np.concatenate([caseD_BH_BL_DB[:idxD + 1], caseB_BH_BL_DB[idxE:idxD + 1][::-1], caseE_BH_BL_DB[idxE:]])
lmix_BL_BH_DB  = np.concatenate([caseD_BL_BH_DB[:idxD + 1], caseB_BL_BH_DB[idxE:idxD + 1][::-1], caseE_BL_BH_DB[idxE:]])
lmix_BL_DB_BH  = np.concatenate([caseD_BL_DB_BH[:idxD + 1], caseB_BL_DB_BH[idxE:idxD + 1][::-1], caseE_BL_DB_BH[idxE:]])
lmix_avg_price = ph * lmix_x + pl * (1 - lmix_x)
lmix_inf       = 3/2 - lmix_x
lmix_phh       = np.array([1 for i in lmix_n])
lmix_phl       = 2 * lmix_x - 1
lmix_pbhh      = lmix_BH_BL_DB + lmix_BL_BH_DB
lmix_pbhb      = lmix_BH_BL_DB
lmix_pbll      = np.array([1 for i in lmix_n])

#----------high-type mixes----------#

k_threshold3 = (np.power(x3, n-1) * (1 - x3)) / (1 - np.power(x3, n))

caseEindicator = (k_threshold3 <= kh) & (kh < 1)
caseE_x = np.array([x3 for i in n])
caseE_BH_BL_DB_numer = (kh * x3 * (1 - np.power(x3, n)) - (1 - x3) * np.power(x3, n))
caseE_BH_BL_DB_denom = ((1 - x3 + kh * x3) * (1 - np.power(x3, n) - np.power(1-x3, n)))
caseE_BH_BL_DB = np.divide(caseE_BH_BL_DB_numer, caseE_BH_BL_DB_denom, where=caseE_BH_BL_DB_denom != 0)
caseE_BH_BL_DB[caseE_BH_BL_DB_denom == 0] = np.nan
caseE_BL_BH_DB = 1 - caseE_BH_BL_DB
caseE_BL_DB_BH = np.array([0 for i in n])

caseBindicator = (0 < kh) & (kh <= k_threshold3)
caseB_x = findx(n, kh)
caseB_BH_BL_DB = np.array([0 for i in n])
caseB_BL_BH_DB = np.array([1 for i in n])
caseB_BL_DB_BH = np.array([0 for i in n])

# combining cases into one array for graphing
idxE = -1 if caseEindicator[-1] == 0 else np.where(caseEindicator > 0)[0][0]
hmix_n         = n
hmix_x         = np.concatenate([caseB_x[:idxE], caseE_x[idxE:]])
hmix_BH_BL_DB  = np.concatenate([caseB_BH_BL_DB[:idxE], caseE_BH_BL_DB[idxE:]])
hmix_BL_BH_DB  = np.concatenate([caseB_BL_BH_DB[:idxE], caseE_BL_BH_DB[idxE:]])
hmix_BL_DB_BH  = np.concatenate([caseB_BL_DB_BH[:idxE], caseE_BL_DB_BH[idxE:]])
hmix_avg_price = ph * hmix_x + pl * (1 - hmix_x)
hmix_inf       = 1/2 + hmix_x
hmix_phh       = 2 * hmix_x
hmix_phl       = np.array([0 for i in hmix_n])
hmix_pbhh      = hmix_BH_BL_DB + hmix_BL_BH_DB
hmix_pbhb      = hmix_BH_BL_DB
hmix_pbll      = np.array([1 for i in hmix_n])

#----------pooling----------#

pool_n         = n
pool_avg_price = np.array([pl  for i in pool_n])
pool_inf       = np.array([0.5 for i in pool_n])

#--------------------Choice probabilities and informativeness in each Nash--------------------#

## n = 1
idx1 = np.where(lmix_n == 1)[0][0]
lmix_pi_1s = np.array([x for y in [lmix_pbhh, lmix_pbll, lmix_phh, lmix_phl] for x in [y[idx1], 1-y[idx1]]])
pool_pi_1s = np.array([np.nan, np.nan, 1, 0, 0, 1, 0, 1]) #pibhh is free, taking values in [0, 1/3]

## n = 2
idx2 = np.where(lmix_n == 2)[0][0]
idx3 = np.where(hmix_n == 2)[0][0]
lmix_pi_2s = np.array([lmix_pbhh[idx2],1-lmix_pbhh[idx2],lmix_pbll[idx2],1-lmix_pbll[idx2],lmix_pbhb[idx2],1-lmix_pbhb[idx2],0,lmix_phh[idx2],1-lmix_phh[idx2],lmix_phl[idx2],1-lmix_phl[idx2]])
hmix_pi_2s = np.array([hmix_pbhh[idx3],1-hmix_pbhh[idx3],hmix_pbll[idx3],1-hmix_pbll[idx3],hmix_pbhb[idx3],1-hmix_pbhb[idx3],0,hmix_phh[idx3],1-hmix_phh[idx3],hmix_phl[idx3],1-hmix_phl[idx3]])
pool_pi_2s = np.array([np.nan, np.nan, 1, 0, 0, 1, 0, 0, 1, 0, 1]) #Here pibhh is free in [0, 1]

inf_lmix_1s = lmix_inf[lmix_n == 1][0]
inf_pool_1s = pool_inf[pool_n == 1][0]
inf_lmix_2s = lmix_inf[lmix_n == 2][0]
inf_hmix_2s = hmix_inf[hmix_n == 2][0]
inf_pool_2s = pool_inf[pool_n == 2][0]

np.savez('../output/Theory Section/eqa.npz', lmix_pi_1s=lmix_pi_1s, pool_pi_1s=pool_pi_1s, lmix_pi_2s=lmix_pi_2s, hmix_pi_2s=hmix_pi_2s, pool_pi_2s=pool_pi_2s)
print('Data written to eqa.npz')

np.savez('../output/Theory Section/inf.npz', inf_lmix_1s=inf_lmix_1s, inf_pool_1s=inf_pool_1s, inf_lmix_2s=inf_lmix_2s, inf_hmix_2s=inf_hmix_2s, inf_pool_2s=inf_pool_2s)
print('Data written to inf.npz')


Running Theory Section
Data written to eqa.npz
Data written to inf.npz


In [14]:
#--------------------Graphs for Theory Section--------------------#

import csv

#--------------------Timing graph (fig. 1)--------------------#

tex_code = r"""
\begin{figure}[htbp]
\tikzstyle{startstop} = [rectangle, rounded corners, minimum width=3cm, minimum height=1cm, text centered, text width=3cm, draw=black, fill=blue!20]
\tikzstyle{arrow}=[draw, -latex]
\begin{tikzpicture}[node distance=2cm]
\node (start) [startstop] {$n$ firms privately realize their qualities $v \in \{v_L, v_H\}$.};
\node (pro2b) [startstop, right of=start, xshift=3cm] {Firms simultaneously choose prices $p \in \{p_L, p_H\}$.};
\node (pro2c) [startstop, right of=pro2b, xshift=3cm] {Representative consumer decides which product to buy, if any.};
\draw [arrow] (start) -- (pro2b);
\draw [arrow] (pro2b) -- (pro2c);
\end{tikzpicture}
\caption{Timing of the Game}
\label{timing}
\end{figure}
"""
with open("../output/Theory Section/timing.tex", "w") as file:
    file.write(tex_code)
print("timing.tex created")

#--------------------Belief Space graph (fig. 2)--------------------#

tex_code = r"""
\begin{figure}[htbp]
\begin{tikzpicture}[scale=7]
\draw[thick,->] (-0.05,0) -- (1.15,0) node[anchor=west] {$\mu_L$};
\draw[thick] (0,-0.05) -- (0,0.08);
\draw[thick,->] (0,0.1067) -- (0,1.15) node[anchor=south] {$\mu_H$};
\draw[thick] (-0.05,1)--(1,1);
\draw[thick] (1,1)--(1,-0.05);
\draw[thick] (-0.05,0.2)--(1,0.2);
\draw[thick] (0,0.3)--(0.7,1);
\draw[thick] (-0.05,0.3)--(0.05,0.3);

\node[left] at (-0.05,0) {0};
\node[left] at (-0.05,1) {1};
\node[below] at (0,-0.05) {0};
\node[below] at (1,-0.05) {1};
\node[left] at (-0.05,0.2) {$\frac{p_H-v_L}{v_H-v_L}$};
\node[left] at (-0.05,0.3) {$\frac{p_H-p_L}{v_H-v_L}$};

\node[right] at (0.035,0.9) {$C: B_H \succ B_L \succ \emptyset$};
\node[right] at (0.36,0.49) {$B: B_L \succ B_H \succ \emptyset$};
\node[right] at (0.23,0.1) {$A: B_L \succ \emptyset \succ B_H$};
\draw[decorate,decoration={zigzag,amplitude=2pt,segment length=4pt},thick] (0,0.08) -- (0,0.1067);
\end{tikzpicture}
\caption{Buyer Belief Space}
\label{belief_space}
\end{figure}
"""
with open("../output/Theory Section/belief_space.tex", "w") as file:
    file.write(tex_code)
print("belief_space.tex created")

#--------------------Seller Strat Space graph (fig. 3)--------------------#

tex_code = r"""
\begin{figure}[htbp]
\begin{tikzpicture}[scale=7]

\draw[thick,->] (-0.05,0) -- (1.15,0) node[anchor=west] {$\mathbb{P}(p_H|v_L)$};
\draw[thick,->] (0,-0.05) -- (0,1.15) node[anchor=south] {$\mathbb{P}(p_H|v_H)$};

\draw[thick] (-0.05,1)--(1,1);
\draw[thick] (1,1)--(1,-0.05);

\node[left] at (-0.05,0) {0};
\node[left] at (-0.05,1) {1};
\node[below] at (0,-0.05) {0};
\node[below] at (1,-0.05) {1};

\node[left] at (-0.1,1) {\shortstack{Type \\ Dependence}};
\node[right] at (1,1) {\shortstack{Pooling \\ at $p_H$}};
\node[below] at (1.1,-0.1) {\shortstack{Reverse Type \\ Dependence}};
\node[below] at (-0.1,-0.1) {\shortstack{Pooling \\ at $p_L$}};

\draw[dotted] (0,0)--(1,1);
\coordinate (P) at (0.54,0.46);
\node[rotate=45] (N) at (P) {\emph{uninformative}};

\draw[line width=0.2cm, dashed, blue] (0,0)--(0,1)--(1,1);
\draw[line width=0.2cm, dashed, blue] (1.2,0.8)--(1.35,0.8);
\node[right] at (1.35,0.8) {\shortstack{Strategies consistent \\ with single-crossing}};

\end{tikzpicture}
\caption{Seller Strategy Space}
\label{seller_strat_space}
\end{figure}
"""

with open("../output/Theory Section/seller_strat_space.tex", "w") as file:
    file.write(tex_code)
print("seller_strat_space.tex created")

#--------------------Average Price graph (fig. 4)--------------------#

output_file = '../output/Theory Section/avgprice.data'
with open(output_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['pool_n', 'pool_avg_price', 'lmix_n', 'lmix_avg_price', 'hmix_n', 'hmix_avg_price'])
    for row in zip(pool_n, pool_avg_price, lmix_n, lmix_avg_price, hmix_n, hmix_avg_price):
        writer.writerow(row)
print(f'Data written to {output_file}')

tex_code = r"""
\begin{figure}[htbp]
\begin{tikzpicture}
\begin{axis}[no markers, legend style={at={(1.1,0.5)}, anchor=west, legend columns=1}, xlabel={Number of firms}, ylabel={Average price in equilibrium}, legend cell align={left}, xtick={1, 2, 3}, xticklabels={1, 2, 3}, ytick={<<pl>>, <<ph>>}, yticklabels={$p_L$, $p_H$}, ultra thick, xmin=0.75, xmax=3.25]
\addplot [color=black, solid] table [x=pool_n, y=pool_avg_price, col sep=comma, unbounded coords=jump] {../output/Theory Section/avgprice.data};
\addlegendentry{Pooling at $p_L$}
\addplot [color=red, dotted] table [x=lmix_n, y=lmix_avg_price, col sep=comma, unbounded coords=jump] {../output/Theory Section/avgprice.data};
\addlegendentry{Low-type mixes}
\addplot [color=blue, dashed] table [x=hmix_n, y=hmix_avg_price, col sep=comma, unbounded coords=jump] {../output/Theory Section/avgprice.data};
\addlegendentry{High-type mixes}
\end{axis}
\end{tikzpicture}
\caption{Average price as competition increases}
\label{average_price}
\begin{minipage}{0.8\textwidth}
\footnotesize
\textit{Notes:} This figure is based on the actual parameter values used in the experiment. Because firms are symmetric, the average price is simply $x p_H + (1-x) p_L$ where $x$ is the unconditional probability of an individual firm setting the high price.
\end{minipage}
\end{figure}
"""
replacements = {"<<ph>>": str(ph), 
                "<<pl>>": str(pl)}
for placeholder, value in replacements.items():
    tex_code = tex_code.replace(placeholder, value)
with open("../output/Theory Section/average_price.tex", "w") as file:
    file.write(tex_code)
print("average_price.tex created")
    
#--------------------Seller Strat Space 2 graph (fig. 5)--------------------#

eqa = np.load('../output/Theory Section/eqa.npz')
loc_n1_lmix_eq = (eqa['lmix_pi_1s'][6], eqa['lmix_pi_1s'][4])
loc_n2_lmix_eq = (eqa['lmix_pi_2s'][9], eqa['lmix_pi_2s'][7])
loc_n2_hmix_eq = (eqa['hmix_pi_2s'][9], eqa['hmix_pi_2s'][7])
loc_n2_pool_eq = (eqa['pool_pi_2s'][9], eqa['pool_pi_2s'][7])

tex_code = r"""
\begin{figure}[htbp]
\begin{tikzpicture}[scale=7]

\draw[thick,->] (-0.05,0) -- (1.15,0) node[anchor=west] {$\mathbb{P}(p_H|v_L)$};
\draw[thick,->] (0,-0.05) -- (0,1.15) node[anchor=south] {$\mathbb{P}(p_H|v_H)$};

\draw[thick] (-0.05,1)--(1,1);
\draw[thick] (1,1)--(1,-0.05);

\node[left] at (-0.05,0) {0};
\node[left] at (-0.05,1) {1};
\node[below] at (0,-0.05) {0};
\node[below] at (1,-0.05) {1};

\node[left] at (-0.1,1) {\shortstack{Type \\ Dependence}};
\node[right] at (1,1) {\shortstack{Pooling \\ at $p_H$}};
\node[below] at (1.1,-0.1) {\shortstack{Reverse Type \\ Dependence}};
\node[below] at (-0.1,-0.1) {\shortstack{Pooling \\ at $p_L$}};

\draw[dotted] (0,0)--(1,1);
\draw[dotted] (0,0.2)--(0.8,1);
\draw[dotted] (0,0.4)--(0.6,1);
\draw[dotted] (0,0.6)--(0.4,1);
\draw[dotted] (0,0.8)--(0.2,1);
\draw[dotted] (0.2,0)--(1,0.8);
\draw[dotted] (0.4,0)--(1,0.6);
\draw[dotted] (0.6,0)--(1,0.4);
\draw[dotted] (0.8,0)--(1,0.2);
\draw[<->] (0.73,0.67)--(0.67,0.73);
\draw[<->] (0.33,0.27)--(0.27,0.33);
\draw[->] (0.43,0.57)--(0.37,0.63);
\draw[->] (0.57,0.43)--(0.63,0.37);
\coordinate (P) at (0.5,0.5);
\node[rotate=45] (N) at (P) {\emph{\shortstack{iso-info \\ lines}}};

\node[draw, circle, fill=black, inner sep=0pt, minimum size=4pt] at <<loc_n1_lmix_eq>> {};
\node[draw, circle, fill=black, inner sep=0pt, minimum size=4pt] at <<loc_n2_lmix_eq>> {};
\node[draw, circle, fill=black, inner sep=0pt, minimum size=4pt] at <<loc_n2_hmix_eq>> {};
\node[draw, circle, fill=black, inner sep=0pt, minimum size=4pt] at <<loc_n2_pool_eq>> {};

\draw[line width=0.1cm, dashed, blue,->] <<loc_n1_lmix_eq>>--<<loc_n2_lmix_eq>>;
\draw[line width=0.1cm, dashed, blue,->] <<loc_n1_lmix_eq>>--<<loc_n2_hmix_eq>>;
\draw[line width=0.1cm, dashed, blue,->] <<loc_n1_lmix_eq>>--<<loc_n2_pool_eq>>;

\end{tikzpicture}
\caption{Potential Seller Strategies as $n$ Increases}
\label{seller_strat_space_2}
\end{figure}
"""

replacements = {"<<loc_n1_lmix_eq>>": str(loc_n1_lmix_eq), 
                "<<loc_n2_lmix_eq>>": str(loc_n2_lmix_eq), 
                "<<loc_n2_hmix_eq>>": str(loc_n2_hmix_eq), 
                "<<loc_n2_pool_eq>>": str(loc_n2_pool_eq)}
for placeholder, value in replacements.items():
    tex_code = tex_code.replace(placeholder, value)
with open("../output/Theory Section/seller_strat_space_2.tex", "w") as file:
    file.write(tex_code)
print("seller_strat_space_2.tex created")
    
#--------------------Informativeness graph (fig. 6)--------------------#

output_file = '../output/Theory Section/informativeness.data'
with open(output_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['pool_n', 'pool_inf', 'lmix_n', 'lmix_inf', 'hmix_n', 'hmix_inf'])
    for row in zip(pool_n, pool_inf, lmix_n, lmix_inf, hmix_n, hmix_inf):
        writer.writerow(row)
print(f'Data written to {output_file}')

tex_code = r"""
\begin{figure}[htbp]
\begin{tikzpicture}
\begin{axis}[no markers, legend style={at={(1.1,0.5)}, anchor=west, legend columns=1}, xlabel={Number of firms}, ylabel={Price informativeness}, y label style={at={(axis description cs:-0.2,0.5)}, anchor=south}, legend cell align={left}, xtick={1, 2, 3}, xticklabels={1, 2, 3}, ytick={0.5, 1}, yticklabels={50\%, 100\%}, ultra thick, xmin=0.75, xmax=3.25, ymin=0.4, ymax=1.1]

\addplot [color=black, solid] table [x=pool_n, y=pool_inf, col sep=comma, unbounded coords=jump] {../output/Theory Section/informativeness.data};
\addlegendentry{Pooling at $p_L$}
\addplot [color=red, dotted] table [x=lmix_n, y=lmix_inf, col sep=comma, unbounded coords=jump] {../output/Theory Section/informativeness.data};
\addlegendentry{Low-type mixes}
\addplot [color=blue, dashed] table [x=hmix_n, y=hmix_inf, col sep=comma, unbounded coords=jump] {../output/Theory Section/informativeness.data};
\addlegendentry{High-type mixes}

\end{axis}
\end{tikzpicture}
\caption{Price informativeness as competition increases}
\label{informativeness}
\begin{minipage}{0.8\textwidth}
\footnotesize
\textit{Note:} This figure is created with the parameters used in the experiment. There always exists an $N \in \mathbb{N}$ such that informativeness has fully converged for all $n \geq N$; this $N$ is always greater than 1, but can be made arbitrarily large by the choice of parameters.
\end{minipage}
\end{figure}
"""

with open("../output/Theory Section/informativeness.tex", "w") as file:
    file.write(tex_code)
print("informativeness.tex created")


timing.tex created
belief_space.tex created
seller_strat_space.tex created
Data written to ../graphs/theory section/avgprice.data
average_price.tex created
seller_strat_space_2.tex created
Data written to ../graphs/theory section/informativeness.data
informativeness.tex created


In [ ]:
#--------------------Basins of attraction for seller strategies--------------------#

tex_code = r"""
\begin{figure}[htbp]\flushleft
\begin{subfigure}[b]{0.45\textwidth}
\begin{tikzpicture}[xscale=3.8,yscale=1.8]
\draw[thick,->] (-0.05,0) node[anchor=east] {0} -- (1.15,0);
\node[anchor=north] at (0.5,-0.3) {$\mathbb{P}(\text{other firms set $p_H$})$};
\draw[thick,->] (0,-0.05) node[anchor=north] {0} -- (0,2.15);
\node[anchor=south] at (0,2.2) {$\mathbb{P}(\text{buyers buy at $p_H$})$};
\draw[thick] (0.05,2) -- (-0.05,2) node[anchor=east] {\makecell{whenever\\$p_H$ exists}};
\draw[thick] (0.05,1) -- (-0.05,1) node[anchor=east] {\makecell{when no\\$p_L$ exists}};
\draw[thick] (1,0.05) -- (1,-0.05) node[anchor=north] {1};
\draw[dotted] (0.05,2) -- (1,2) -- (1,0.05);
\draw[dotted] (0.05,1) -- (1,1);
\draw[domain=0:1, smooth, variable=\x, blue, thick] plot ({\x},{1/3});
\fill[blue!30, opacity=0.6] (0,0) -- (0,1/3) -- (1,1/3) -- (1,0) -- cycle;
\fill[red!30, opacity=0.6] (0,1/3) -- (1,1/3) -- (1,2) -- (0,2) -- cycle;
\node[anchor=center] at (0.5,1/6) {\textcolor{blue}{Set $p_L$}};
\node[anchor=center] at (0.5,1.5) {\textcolor{red}{Set $p_H$}};
\node[draw, star, fill=black, star point ratio=2.25, inner sep=0pt, minimum size=8pt] at (2/3,1/2) {};
\end{tikzpicture}
\caption{$n = 1$}
\end{subfigure}
\hspace{0.01\textwidth}
\begin{subfigure}[b]{0.4\textwidth}
\begin{tikzpicture}[xscale=3.8,yscale=1.8]
\draw[thick,->] (-0.05,0) node[anchor=east] {0} -- (1.15,0);
\node[anchor=north] at (0.5,-0.3) {$\mathbb{P}(\text{other firms set $p_H$})$};
\draw[thick,->] (0,-0.05) node[anchor=north] {0} -- (0,2.15);
\node[anchor=south] at (0,2.2) {$\mathbb{P}(\text{buyers buy at $p_H$})$};
\draw[thick] (0.05,2) -- (-0.05,2) node[anchor=east] {\makecell{whenever\\$p_H$ exists}};
\draw[thick] (0.05,1) -- (-0.05,1) node[anchor=east] {\makecell{when no\\$p_L$ exists}};
\draw[thick] (1,0.05) -- (1,-0.05) node[anchor=north] {1};
\draw[dotted] (0.05,2) -- (1,2) -- (1,0.05);
\draw[dotted] (0.05,1) -- (1,1);
\draw[domain=0:1, smooth, variable=\x, blue, thick] plot ({\x},{1/3});
\draw[domain=0.5:1, smooth, variable=\x, blue, thick] plot ({\x},{(1/3)*(1 + 1/\x)});
\draw[domain=0:0.5, smooth, variable=\x, blue, thick] plot ({\x},{1 + (1/3-(2/3)*\x)/(2-(4/3)*\x)});
\fill[blue!30, opacity=0.6] (0,0) -- plot[domain=0:0.5, smooth] ({\x},{1 + (1/3-(2/3)*\x)/(2-(4/3)*\x)}) -- plot[domain=0.5:1, smooth] ({\x},{(1/3)*(1 + 1/\x)}) -- (1,0) -- cycle;
\fill[red!30, opacity=0.6] (0,2) -- plot[domain=0:0.5, smooth] ({\x},{1 + (1/3-(2/3)*\x)/(2-(4/3)*\x)}) -- plot[domain=0.5:1, smooth] ({\x},{(1/3)*(1 + 1/\x)}) -- (1,2) -- cycle;
\node[anchor=center] at (0.5,0.6) {\textcolor{blue}{Set $p_L$}};
\node[anchor=center] at (0.5,1.5) {\textcolor{red}{Set $p_H$}};
\node[draw, star, fill=black, star point ratio=2.25, inner sep=0pt, minimum size=8pt] at (0.375,1.055555) {};
\node[draw, star, fill=black, star point ratio=2.25, inner sep=0pt, minimum size=8pt] at (0.625,1.136363) {};
\end{tikzpicture}
\caption{$n = 2$}
\end{subfigure}
\caption{Basins of Attraction for High-quality Seller}
\label{basin_high}
\end{figure}
"""

with open("../output/Theory Section/basin_high.tex", "w") as file:
    file.write(tex_code)
print('Created basin_high.tex')
print('\n')


In [ ]:
# Additional Calculations

#lowmix_muL       = np.array([0 for i in lowmix_n])
#lowmix_muH       = 1 / (2 * lowmix_x)
#lowmix_pr_sale_h = ((lowmix_BL_BH_DB * np.power(lowmix_x, lowmix_n - 1) 
#                     + lowmix_BH_BL_DB * ((1 - np.power(1 - lowmix_x, lowmix_n)) / lowmix_x)) / lowmix_n)
#lowmix_pr_sale_l = ((lowmix_BH_BL_DB * np.power(1 - lowmix_x, lowmix_n - 1) 
#                     + (1 - lowmix_BH_BL_DB) * ((1 - np.power(lowmix_x, lowmix_n)) / (1 - lowmix_x))) / lowmix_n)
#lowmix_high_prof = lowmix_phh * lowmix_pr_sale_h * (ph - ch) + (1 - lowmix_phh) * lowmix_pr_sale_l * (pl - ch)
#lowmix_low_prof  = lowmix_phl * lowmix_pr_sale_h * (ph - cl) + (1 - lowmix_phl) * lowmix_pr_sale_l * (pl - cl)
#lowmix_sell_prof = 0.5 * lowmix_high_prof + 0.5 * lowmix_low_prof
#lowmix_buy_prof  = (np.power(lowmix_x, lowmix_n) * lowmix_pbhh * (lowmix_muH * (vh - vl) - (ph - vl))
#                    + np.power(1 - lowmix_x, lowmix_n) * (lowmix_muL * (vh - vl) + (vl - pl))
#                    + (1 - np.power(lowmix_x, lowmix_n) - np.power(1 - lowmix_x, lowmix_n)) 
#                        * (lowmix_pbhb * (lowmix_muH * (vh - vl) - (ph - vl))
#                           + (1 - lowmix_pbhb) * (lowmix_muL * (vh - vl) + (vl - pl))))

#highmix_muL       = (1 - 2 * highmix_x) / (2 - 2 * highmix_x)
#highmix_muH       = np.array([1 for i in highmix_n])
#highmix_pr_sale_h = ((highmix_BL_BH_DB * np.power(highmix_x, highmix_n - 1) 
#                     + highmix_BH_BL_DB * ((1 - np.power(1 - highmix_x, highmix_n)) / highmix_x)) / highmix_n)
#highmix_pr_sale_l = ((highmix_BH_BL_DB * np.power(1 - highmix_x, highmix_n - 1) 
#                     + (1 - highmix_BH_BL_DB) * ((1 - np.power(highmix_x, highmix_n)) / (1 - highmix_x))) / highmix_n)
#highmix_high_prof = highmix_phh * highmix_pr_sale_h * (ph - ch) + (1 - highmix_phh) * highmix_pr_sale_l * (pl - ch)
#highmix_low_prof  = highmix_phl * highmix_pr_sale_h * (ph - cl) + (1 - highmix_phl) * highmix_pr_sale_l * (pl - cl)
#highmix_sell_prof = 0.5 * highmix_high_prof + 0.5 * highmix_low_prof
#highmix_buy_prof  = (np.power(highmix_x, highmix_n) * highmix_pbhh * (highmix_muH * (vh - vl) - (ph - vl))
#                     + np.power(1 - highmix_x, highmix_n) * (highmix_muL * (vh - vl) + (vl - pl))
#                     + (1 - np.power(highmix_x, highmix_n) - np.power(1 - highmix_x, highmix_n)) 
#                          * (highmix_pbhb * (highmix_muH * (vh - vl) - (ph - vl))
#                             + (1 - highmix_pbhb) * (highmix_muL * (vh - vl) + (vl - pl))))

#pool_x         = np.array([0   for i in pool_n])
#pool_phh       = np.array([0   for i in pool_n])
#pool_phl       = np.array([0   for i in pool_n])
#pool_muL       = np.array([0.5 for i in pool_n])
#pool_BH_BL_DB  = np.array([0   for i in pool_n])
#pool_pr_sale_l = 1 / pool_n
#pool_high_prof = pool_pr_sale_l * (pl - ch)
#pool_low_prof  = pool_pr_sale_l * (pl - cl)
#pool_sell_prof = 0.5 * pool_high_prof + 0.5 * pool_low_prof
#pool_buy_prof  = pool_muL * (vh - vl) + (vl - pl)
#pool_muH \in [0,1]
#pool_BL_BH_DB \in [0,1]
#pool_BL_DB_BH = 1 - pool_BL_BH_DB
#pool_pr_sale_h = ((pool_BL_BH_DB * np.power(pool_x, pool_n - 1) 
#                   + pool_BH_BL_DB * ((1 - np.power(1 - pool_x, pool_n)) / pool_x)) / pool_n)
